# 손글씨이미지데이터(MNIST)를 이용한 예측시스템

- MNIST(바이너리데이터)
- 바이너리 데이터를 디코딩하여(디코딩 처리법)
- 샘플 데이터(전체 데이터의 일부분)을 이용하여 머신러닝 모델을 구축
- 예측 수행 (64%정도로 나올듯)
- 머신러닝 시험(내일) 테스트 => 정확도를 높이는것

# 연구목표

- 손글씨 이미지(0-9)를 학습시켜서, 새로운 손글씨 이미지가 들어오면 0-9중 어떤것인지 예측한다
- 원본 데이터를 디코딩하여(바이너리 데이터) 개별 이미지로 저장
- 픽셀 데이터를 백터화 처리
- 정확도를 94% 이상 높이는 것

# 데이터 획득/수집

- http://yann.lecun.com/exdb/mnist/
- 1. 사이트를 긁어서(웹스크래핑)
- 2. 데이터의 URL을 직접 획득
- 3. 로컬에 바로 저장 (URL을 직접 연결하여 로컬 PC에 파일로 원하는 위치에 저장)

In [1]:
import urllib.request as req
from bs4 import BeautifulSoup

target_url  = 'http://yann.lecun.com/exdb/mnist/'
# 요청
res         = req.urlopen( target_url )
# 파싱
soup        = BeautifulSoup( res, 'html5lib' )
soup

<html><head>
   <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
   <title>MNIST handwritten digit database, Yann LeCun, Corinna Cortes and Chris Burges</title>
<script type="text/javascript">
var gaJsHost = (("https:" == document.location.protocol) ? "https://ssl." : "http://www.");
document.write(unescape("%3Cscript src='" + gaJsHost + "google-analytics.com/ga.js' type='text/javascript'%3E%3C/script%3E"));
</script><script type="text/javascript">
var pageTracker = _gat._getTracker("UA-6178702-1");
pageTracker._trackPageview();
</script></head>




<body alink="#111111" background="wm5b.gif" bgcolor="#FFFFFF" link="#FF0000" vlink="#820594">

<center>
<h1>
<font color="#FF0000"><font size="+4">THE MNIST DATABASE</font></font></h1></center>

<center><font color="#FF0000"><font size="+4">of handwritten digits</font></font></center>

<center><font color="#FF0000"><font size="+4"></font></font></center>

<center><font color="#3366FF"><font size="+1"><a href="http://yann

In [2]:
# URL이 <tt> 태그에 존재하고, 최초 4개까지가 원하는 <tt> 이다
for tt in soup.find_all('tt')[:4]:
  print( target_url + tt.a.string )

http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz


In [3]:
# 다음 단계를 위해서 최종 URL이 담긴 리스트로 전달하겠다
# files
files = [ tt.a.string 
          for tt in soup.find_all('tt')[:4] ]
files

['train-images-idx3-ubyte.gz',
 'train-labels-idx1-ubyte.gz',
 't10k-images-idx3-ubyte.gz',
 't10k-labels-idx1-ubyte.gz']

# 데이터 준비

- 압축해제
- 디코딩
- 백터화

In [0]:
import os, os.path, gzip

In [0]:
# 파일 저장 위치 지정
savePath = './data/mnist'
# 해당 디렉토리가 없다면
if not os.path.exists( savePath ):
  # 디랙토리를 만들어라 
  os.makedirs( savePath )

In [6]:
from tqdm import tqdm_notebook
# 저장
for file in tqdm_notebook(files):
  # 로컬에 저장위치
  # ./data/mnist/train-images-idx3-ubyte.gz
  #print( '%s/%s' % (savePath ,file) )
  #print( f'{savePath}/{file}')
  local_path = '%s/%s' % (savePath ,file)
  if not os.path.exists( local_path ):
    # 원격지에 리소스를 로컬상 PC에 바로 읽어서 저장
    req.urlretrieve( target_url+file, local_path)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
# 압축해제
for file in tqdm_notebook(files):
  # 원본 파일 (*.gz)
  ori_path = '%s/%s' % (savePath , file)
  # 압축해제 파일 (*) <= .gz 제거
  raw_path = '%s/%s' % (savePath , file[:-3])
  # 파일 오픈 -> 기록
  with gzip.open( ori_path, 'rb' ) as fg:
    # 대용량이면 분할해서, 소용량이니까 그냥 전체 읽기
    tmp = fg.read()
    with open( raw_path, 'wb' ) as f:
      f.write( tmp )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [0]:
# 데이터중 이미지 데이터의 크기는 
# 4 + 4 + 4 + 4 + 60000*28*28 (byte)
# 데이터중 레이블 크기는 
# 4 + 4 + 60000 (byte)

## label 파일의 구조
- 구조
  > magic number : 4byte  
  > label 개수 : 4byte  
  > label : 나머지~(1byte씩)  

## image 파일의 구조
- 구조
  > magic number : 4byte  
  > image 개수 : 4byte  
  > 이미지 가로 길이 : 4byte  
  > 이미지의 세로 길이 : 4byte  
  > 이미지 : 28*28(=784) byte가 image 개수만큼 반복  

## 정수(integer) 처리
- 정수값을 읽을때, 앞에서부터부터 읽을것인가, 뒤에서부터 읽을것인가?
- 오더링(ordering)
- 빅 에디언

- 값 : 0x12345678

|메모리주소|0x100|0x101|0x102|0x103|
|:--:|:--:|:--:|:--:|:--:|
|값|0x12|0x34|0x56|0x78|

- 리틀 에디언

- 값 : 0x12345678

|메모리주소|0x100|0x101|0x102|0x103|
|:--:|:--:|:--:|:--:|:--:|
|값|0x78|0x56|0x34|0x12|

## 규격에서 의해서 본 파일은 

- high endian(빅 엔디안으로 저장)

In [0]:
# 바이너리 데이터를 읽을때 사용한 모듈
import struct 

In [10]:
def decode_mnist( dataType='train', dir='/content/data/mnist', samples=1000 ):
    
    label_name = f'{dir}/{dataType}-labels-idx1-ubyte'  
    image_name = f'{dir}/{dataType}-images-idx3-ubyte'
    print(label_name, '\n', image_name)
    
    label_f = open( label_name, 'rb')
    image_f = open( image_name, 'rb')
    
    csv_f = open( f'{dir}/{dataType}.csv', 'w', encoding='utf-8' )

    LABEL_HEAD_SIZE = 4 + 4
    magic, label_cnt = struct.unpack('>II', label_f.read(LABEL_HEAD_SIZE) )
    print(magic, label_cnt)

    IMAGE_HEAD_SIZE = 4 + 4 + 4 + 4
    magic_img, image_cnt, row, col = struct.unpack('>IIII', image_f.read(IMAGE_HEAD_SIZE) )
    print(magic_img, image_cnt, row, col)
    pixels = row * col
    

    for idx in range(label_cnt):
        if idx >= samples:
            break
    
        label_value = struct.unpack( 'B', label_f.read(1) )    
        label = label_value[0] # (5,) => 0번째만 추출

        binary_data = image_f.read( pixels )
        strPixelData= list( map( lambda x:str(x) , binary_data ) ) 
        csv_f.write( str(label) + ',' )
        csv_f.write( ','.join(strPixelData) + '\n' )
        #break
        if idx == 0: # 1회만 수행
            with open('test.pgm', 'w', encoding='utf-8') as f:
                f.write( 'P2 28 28 255\n' + ' '.join(strPixelData) )


    if label_f:label_f.close()
    if image_f:image_f.close()
    if csv_f:csv_f.close()
        
decode_mnist()

/content/data/mnist/train-labels-idx1-ubyte 
 /content/data/mnist/train-images-idx3-ubyte
2049 60000
2051 60000 28 28


In [0]:
# # # 함수로 파일을 풀어보겟다
# # dataType : train, t10k

# def decode_mnist( dataType='train', dir='./data/mnist' ):
    
#     label_name = f'{dir}/{dataType}-labels-idx1-ubyte'  
#     image_name = f'{dir}/{dataType}-images-idx3-ubyte'
#     print(label_name, '\n', image_name)
    
#     # 파일오픈
#     label_f = open( label_name, 'rb')
#     image_f = open( image_name, 'rb')
    
#     # -----------------------------------------------------------
#     csv_f = open( f'{dir}/{dataType}.csv', 'w', encoding='utf-8' )
#     # -----------------------------------------------------------
#     LABEL_HEAD_SIZE = 4 + 4
#     magic, label_cnt = struct.unpack('>II', label_f.read(LABEL_HEAD_SIZE) )
#     print(magic, label_cnt)
    
#     IMAGE_HEAD_SIZE = 4 + 4 + 4 + 4
#     magic_img, image_cnt, row, col = struct.unpack('>IIII', image_f.read(IMAGE_HEAD_SIZE) )
#     print(magic_img, image_cnt, row, col)
#     pixels = row * col
#     # -----------------------------------------------------------
#     # 파일닫기
#     if label_f:label_f.close()
#     if image_f:image_f.close()
#     if csv_f:csv_f.close()

# decode_mnist()

In [12]:
# 최종 학습용 데이터 준비
decode_mnist( dataType='train', samples = 750) # 훈련용
decode_mnist( dataType='t10k' , samples = 250) # 테스트용

/content/data/mnist/train-labels-idx1-ubyte 
 /content/data/mnist/train-images-idx3-ubyte
2049 60000
2051 60000 28 28
/content/data/mnist/t10k-labels-idx1-ubyte 
 /content/data/mnist/t10k-images-idx3-ubyte
2049 10000
2051 10000 28 28


# 데이터 탐색/분석

- 픽셀 데이터의 정규화 추가
- 훈련에 필요한 데이터의 형태도 준비

In [0]:

def load_csv( dataType='train', dir='/content/data/mnist' ):

  # 0. 데이터를 담는 자료구조
  labels = list()
  images = list()

  # 1. csv 파일 오픈
  with open( f'{dir}/{dataType}.csv', 'r' ) as f:
    # 한줄씩 읽는다
    for line in f:      
      # 분해
      tmp = line.strip().split(',')
      #print(tmp)
      # 정답 데이터 담기 -> 타입은 수치로 변환
      labels.append( int(tmp[0]) )
      # 각 픽셀을 256개(색상의 총수)로 정규화 하여서 리스트로처리
      images.append( list( map( lambda x:int(x)/256, tmp[1:] ) ) )
      #break
  return { 'labels':labels   ,'images':images }

#load_csv()

In [0]:
# # load_csv()
# def load_csv( dataType='train',dir='/content/data/mnist' ):

#     labels = list()
#     images = list()
    
#     with open( f'{dir}/{dataType}.csv', 'r' ) as f:
#         for line in f:      
#             # 분해
#             tmp = line.strip().split(',')
# #             print(tmp)
#             labels.append( int(tmp[0]) )
#             images.append( list( map( lambda x:int(x)/256, tmp[1:] ) ) )
#             #break
#     return { 'labels':labels   ,'images':images }

# # load_csv()

# 데이터 모델링

In [0]:
# sklearn
from sklearn import svm, model_selection, metrics

In [0]:
# 1. 알고리즘 생성
clf = svm.SVC()

In [17]:
# 2. 훈련, 테스트 데이터 준비
train = load_csv()
test  = load_csv( dataType='t10k' )
len(train['labels']), len(test['labels']) # 데이터 준비 완료 


(750, 250)

In [0]:


X_train = train['images']
y_train = train['labels']

X_test = test['images']
y_test = test['labels']

In [19]:
# 3. 학습
# clf.fit( train['images'], train['labels'] )
clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
# 4. 예측
# predict = clf.predict( test['images'] )
predict = clf.predict( X_test )

In [21]:
# 5. 성능평가 정확도 확인
metrics.accuracy_score(y_test, predict )
# metrics.accuracy_score( test['labels'], predict )

0.9

In [22]:
# t = metrics.classification_report( test['labels'], predict )
t = metrics.classification_report( y_test, predict )
print( t )

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        34
           2       0.81      0.92      0.86        24
           3       1.00      0.70      0.82        23
           4       0.91      0.94      0.93        33
           5       0.72      0.84      0.78        25
           6       1.00      0.86      0.93        22
           7       0.81      0.90      0.85        29
           8       0.93      0.93      0.93        14
           9       0.92      0.89      0.91        27

    accuracy                           0.90       250
   macro avg       0.91      0.90      0.90       250
weighted avg       0.91      0.90      0.90       250



# 머신러닝 실기 평가

- 95% 이상 정확도를 높이시오
  - 95%까지는 실기점수 60점
  - 그 이상은 랭킹순으로 가산점 부여
  - 최종 주피터 파일 제줄(구글 드라이브 제출) 
  - 이름_정확도.ipynb
  - 시작은 14:00 시작

In [23]:
# 세트 전체
decode_mnist( dataType='train', samples = 60000) 
decode_mnist( dataType='t10k' , samples = 10000) 

/content/data/mnist/train-labels-idx1-ubyte 
 /content/data/mnist/train-images-idx3-ubyte
2049 60000
2051 60000 28 28
/content/data/mnist/t10k-labels-idx1-ubyte 
 /content/data/mnist/t10k-images-idx3-ubyte
2049 10000
2051 10000 28 28


In [24]:
# 데이터 준비
train = load_csv()
test  = load_csv( dataType='t10k' )
len(train['labels']), len(test['labels']) # 데이터 준비 완료 


(60000, 10000)

In [0]:
# 담기
X_train = train['images']
y_train = train['labels']

X_test = test['images']
y_test = test['labels']

In [0]:
# 정확도 확인
clf = svm.SVC()
clf.fit(X_train, y_train)
predict = clf.predict( X_test )

In [27]:
metrics.accuracy_score(y_test, predict )

0.9792

In [28]:
# 리포트 작성 
t = metrics.classification_report( y_train, predict )
print( t )

ValueError: ignored

In [0]:
# 여기까지 0.9792

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC 
ro

In [0]:
scaler = MinMaxScaler().fit( X_train )
scaler

In [0]:
X_train_scaled = scaler.transform( X_train )
X_test_scaled = scaler.transform( X_test ) 
X_train_scaled

In [0]:
param_grid = {
  'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000],
  'gamma':[0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

In [0]:
# 학습용 데이터
from sklearn import datasets
# 데이터를 학습용과 테스트용으로 나눌수 있는 함수
from sklearn.model_selection import train_test_split
# 데이터 표준화
from sklearn.preprocessing import StandardScaler
# Perceptron 머신러닝을 위한 클래스
from sklearn.linear_model import Perceptron
# 로지스트 회귀를 위한 클래스 
from sklearn.linear_model import LogisticRegression
# SVM을 위한 클래스 
from sklearn.svm import SVC
# 의사결정 나무를 위한 클래스 
from sklearn.tree import DecisionTreeClassifier
# 랜덤 포레스트
from sklearn.ensemble import RandomForestClassifier
# 정확도 계산을 위한 함수 
from sklearn.metrics import accuracy_score

In [0]:
grid   = GridSearchCV( SVC(), param_grid, cv=5 )
grid_R = RandomForestClassifier(criterion='entropy', n_estimators=10, max_depth=3,n_jobs=2, random_state=0)
grid_L = LogisticRegression(C=1000.0,random_state=0)
grid_S = SVC( kernel='linear', C= 1.0, random_state= 0 )
grid_D = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)

In [0]:
tmp = [grid,grid_R,grid_L,grid_S,grid_D]

for arg in tqdm_notebook(tmp) :
    arg.fit( X_train, y_train )
    grid.best_params_
    grid.best_score_
    arg.score( X_test_scaled, y_test )
    print('=')

In [0]:
grid.best_params_

In [0]:
grid.best_score_

In [0]:
grid.score( X_test_scaled, y_test )

In [0]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [0]:
# 생성 방법 비교
pipe_std = Pipeline( [ ('scaler', MinMaxScaler()),
                        ('classifier', SVC())   
                      ] )

pipe_std_m = make_pipeline( MinMaxScaler(), SVC() )

In [0]:
pipe = Pipeline( [  ('preprocessing', StandardScaler()),
                    ('classifier',    SVC())   
                  ] )

In [0]:
param_grid = [
  {
    'preprocessing':[StandardScaler(), MinMaxScaler()],
    'classifier':[SVC()],
    'classifier__C':[0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'classifier__gamma':[0.001, 0.01, 0.1, 1, 10, 100, 1000]
  },
  {
    'preprocessing':[None],
    'classifier':[RandomForestClassifier(n_estimators=100)],
    'classifier__max_features':[1,2,3]
  }
]

In [0]:
grid = GridSearchCV( pipe, param_grid, cv=5 )

In [0]:
grid.fit( X_train, y_train )

In [0]:
grid.best_params_

In [0]:
grid.best_score_

In [0]:
grid.score( X_test, y_test )